# Setting up Event Markers (Triggers) for Your Task

In many EEG or physiological experiments, it’s crucial to **mark** task events using “triggers” or “event markers.” By sending numeric codes to your recording system, you can synchronize recorded signals with specific experimental conditions or participant responses.

This notebook focuses on **two main approaches** for sending triggers:
1. **Phasic Triggers** – Short pulses at key moments (e.g., stimulus onset, button press).
2. **Tonic Triggers** – Holding a trigger high for the duration of an event (or a longer time window).

We also provide a **Go/No-Go** task example (uploaded to [Pavlovia](https://pavlovia.org/)) to show exactly how you might insert these triggers into your PsychoPy code modules. Although this tutorial highlights **Go/No-Go**, the same principles apply to **any** task needing triggers.


## 1. General Setup

### 1.1 Hardware and Serial Communication

- **Neurospec MMBT-S** acts as a bridge from your computer’s serial port to the DSI system (or other hardware).  
- On Windows, your serial port might be named `COM3`; on Linux/macOS, it might be `/dev/ttyUSB0`.

We’ll use Python’s `serial` (PySerial) library. Typically, you start by:

In [ ]:
import serial

# Open the serial port; adjust 'COM3' to match your system
Ser = serial.Serial('COM3', baudrate=115200, timeout=0.1)

# Immediately reset the trigger line to 0 (idle state)
Ser.write(str.encode(chr(0)))
print("Serial port opened and trigger set to 0.")

### 1.2 Tonic vs. Phasic Triggers

- **Tonic**: Keep a nonzero trigger value “high” for the duration of an event. You manually reset it to 0 at the end.  
- **Phasic**: Send a brief pulse (e.g., 10–50 ms) exactly at the moment of interest, then automatically return to 0.

**Which approach to use?**  
- **Phasic** is typically best for discrete or momentary events (such as stimulus onset or a button press). It ensures precise time-locking and avoids leaving the trigger “on” too long.  
- **Tonic** can represent a continuous state (for example, while a stimulus is displayed), then you reset at the event’s conclusion. This is useful if you need the trigger to remain high throughout an entire condition.

### 1.3 send_trigger vs. send_trigger_onflip

In many PsychoPy experiments, you need to synchronize triggers with visual (or other) stimuli. Specifically, **when** you send the trigger can be critical:

1. **`send_trigger`**:  
   - Sends a short pulse **immediately** upon function call.  
   - It does **not** wait for a screen refresh. Instead, the trigger goes out at whatever point in time the code is executed.  
   - This is perfect if you have an event that isn’t strictly tied to a stimulus flip, or if you want to send a trigger for something that happens asynchronously (like a response event or a condition changing in the background).

2. **`send_trigger_onflip`**:  
   - Schedules the trigger to turn **on** right at the next **screen refresh** (`win.flip()`), holds it for a specified duration, and then resets it to 0.  
   - This is crucial if you want the trigger timestamp to line up exactly with the moment a stimulus appears on screen. Psychophysics and EEG/physiology experiments often require you to send the marker at the **precise** frame when the stimulus is displayed.

### When to Use Which?

- **Use `send_trigger`**:
  - For events that aren’t locked to visual presentation (e.g., subject responses, intermediate states, or sending a marker at an arbitrary time in your experiment flow).  
  - If you just want a quick, blocking pulse that doesn’t depend on `win.flip()` scheduling.

- **Use `send_trigger_onflip`**:
  - When you need the trigger to coincide with the exact moment the screen updates.  
  - Commonly used for marking **stimulus onset**, ensuring your EEG/physiological data lines up with the frame the participant sees the stimulus for the first time.

Below is the **complete code** for both functions, along with docstrings explaining their usage for **phasic** and **tonic** approaches:


### Phasic: 

In [ ]:
from psychopy import core

def send_trigger_phasic(ser, data, dur=0.01):
    """
    Send a short trigger pulse by writing `data` to the serial port, 
    wait approximately `dur` seconds, then write 0 to turn the trigger off.
    
    Args:
        ser (serial.Serial): An open serial port object.
        data (int): The trigger value to send.
        dur (float): Duration in seconds to keep the trigger high.
    """
    # Mark the start time
    start = core.getTime()
    
    # Write the trigger value
    ser.write(str.encode(chr(data)))
    
    # Measure how long that took
    elapsed = core.getTime() - start
    remaining = dur - elapsed
    
    # Avoid negative waiting if sending took longer than dur
    if remaining > 0:
        core.wait(remaining)
    
    # Reset the trigger to 0
    ser.write(str.encode(chr(0)))


def send_trigger_onflip_phasic (win, ser, trigger_value, dur=0.05):
    """
    Turn a trigger on at the next screen flip, hold it for `dur` seconds,
    then reset to 0. This approach doesn't use core.callLater.
    """
    
    # 1) We'll queue the "turn on" for the next flip:
    def turn_trigger_on():
        ser.write(str.encode(chr(trigger_value)))
        print(f"Trigger {trigger_value} sent")
    
    # Queue it to happen on the next flip
    win.callOnFlip(turn_trigger_on)
    
    # 2) Actually flip so that the trigger command executes now
    win.flip()

    # 3) Wait the desired pulse duration
    core.wait(dur)

    # 4) Turn the trigger off
    ser.write(str.encode(chr(0)))
    print("Trigger reset to 0")



### Tonic

In [ ]:
def send_trigger_tonic (ser, trigger_value):
    """
    Send a 'tonic' trigger by writing `trigger_value` to the serial port.
    The line remains at this value until you explicitly reset it.
    
    Args:
        ser (serial.Serial): An open serial port object.
        trigger_value (int): The trigger value to send.
    """
    ser.write(str.encode(chr(trigger_value)))
    print(f"[TONIC] Trigger set to {trigger_value} (must reset later).")


def reset_trigger_tonic (ser):
    """
    Manually reset the trigger line to 0 after using a tonic trigger.
    
    Args:
        ser (serial.Serial): An open serial port object.
    """
    ser.write(str.encode(chr(0)))
    print("[TONIC] Trigger reset to 0.")


def send_trigger_onflip(win, ser, trigger_value):
    """
    Turn a 'tonic' trigger on at the next screen flip (i.e., precisely when the new frame is drawn).
    The line remains at this value until you explicitly reset it by calling `reset_tonic_trigger`.
    
    Args:
        win (visual.Window): PsychoPy window object.
        ser (serial.Serial): An open serial port object.
        trigger_value (int): The trigger value to send.
    """
    def turn_trigger_on():
        ser.write(str.encode(chr(trigger_value)))
        print(f"[TONIC] Trigger {trigger_value} set on next flip (must reset later).")
    
    # Schedule the trigger command for the next screen refresh
    win.callOnFlip(turn_trigger_on)
    
    # Actually flip to execute the scheduled call
    win.flip()


## 2. Example Task: Go/No-Go on Pavlovia

This section demonstrates how to incorporate triggers (both **phasic** and **tonic**) into a **Go/No-Go** experiment. The same logic applies to any other task; simply adapt the code for your events and conditions. We’ve also uploaded an example task to [Pavlovia](https://pavlovia.org/) so you can see the implementation in action.


### 2.1 High-Level Task Flow

A typical **Go/No-Go** experiment might follow these steps:

1. **Task Start**  
   - Optionally send a “task start” trigger, either as a brief (phasic) pulse or a tonic signal.  
2. **Trial Begins**  
   - Display either a **Go** or **No-Go** stimulus.  
   - Immediately mark the stimulus onset with a phasic or tonic trigger.  
3. **Participant Responds** (or not)  
   - Collect keypresses.  
   - Determine correctness (Go correct = pressed, No-Go correct = no press).  
   - Send a trigger to mark correct or incorrect responses.  
4. **Trial Ends**  
   - If using **tonic** triggers, reset the line to 0 here (so it doesn’t carry into the next trial).  
5. **Repeat** for the desired number of trials.  
6. **Task End**  
   - Send a “task end” trigger.  
   - Reset the line to 0, and close the serial port.


### 2.2 Adding Triggers in PsychoPy’s Builder

We’ll assume you have a Builder setup with:
- A routine named **“trial”** for each Go/No-Go stimulus presentation.
- A **keyboard component** named `key_resp` for participant responses.
- A **Code Component** in that routine (plus potentially separate routines for “start_trigger” and “goodbye”).

#### 2.2.1 “Begin Experiment” Tab

In your Code Component’s **“Begin Experiment”** section:


In [ ]:
import serial
from psychopy import core, visual, event

# Initialize the serial port for the Neurospec MMBT-S -> DSI
Ser = serial.Serial('COM3')

# Immediately set trigger line to 0 at the experiment start
Ser.write(str.encode(chr(0)))
print("Serial port opened. Trigger line reset to 0.")

# Define trigger codes (integers for each event)
TASK_START   = 10
GO_ONSET     = 1
NOGO_ONSET   = 2
GO_CORRECT   = 3
NOGO_CORRECT = 4
INCORRECT    = 5
TASK_END     = 99

# Define your phasic/tonic trigger functions
def send_trigger(ser, data, dur=0.01):

    start = core.getTime()
    ser.write(str.encode(chr(data)))
    elapsed = core.getTime() - start
    remaining = dur - elapsed
    if remaining > 0:
        core.wait(remaining)
    ser.write(str.encode(chr(0)))


def send_trigger_onflip(win, ser, trigger_value, dur=0.05):

    def turn_trigger_on():
        ser.write(str.encode(chr(trigger_value)))
        print(f"Trigger {trigger_value} sent")
    win.callOnFlip(turn_trigger_on)
    win.flip()
    core.wait(dur)
    ser.write(str.encode(chr(0)))
    print("Trigger reset to 0")

#### 2.2.2 “Begin Routine” Tab (Stimulus Onset)

In the **“Begin Routine”** tab of your `trial` routine’s Code Component, you can decide how to mark stimulus onset. For instance, if your trial dictionary is named `thisTrial` and has a key `'this_image'`:


In [ ]:
# Example: phasic approach for exact onset
if thisTrial['this_image'] == 'go.png':
    send_trigger_phasic(Ser, GO_ONSET, dur=0.01)
elif thisTrial['this_image'] == 'nogo.png':
    send_trigger_phasic(Ser, NOGO_ONSET, dur=0.01)

# If using tonic instead, do:
# if thisTrial['this_image'] == 'go.png':
#     send_trigger_tonic(Ser, GO_ONSET)
# elif thisTrial['this_image'] == 'nogo.png':
#     send_trigger_tonic(Ser, NOGO_ONSET)

#### 2.2.3 “End Routine” Tab (Response Evaluation)
After the participant responds (or doesn’t), you can mark correct vs. incorrect outcomes. For instance:

In [ ]:
# Grab whatever keys were pressed this trial
resp_keys = key_resp.keys  # Might be [], ['space'], or None

# Ensure resp_keys is always a list (some older PsychoPy versions return None)
if resp_keys is None:
    resp_keys = []

# Determine correctness based on image and response
if thisTrial['this_image'] == 'go.png':
    # For a Go trial, correct if the participant pressed 'space'
    if 'space' in resp_keys:
        send_trigger(Ser,GO_CORRECT_TRIGGER)  
        print(f"Correct Go response, trigger {GO_CORRECT_TRIGGER} sent")
    else:
        send_trigger(Ser,INCORRECT_TRIGGER)
        print(f"Incorrect Go response, trigger {INCORRECT_TRIGGER} sent")

elif thisTrial['this_image'] == 'nogo.png':
    # For a NoGo trial, correct if NO key was pressed
    if len(resp_keys) == 0:
        send_trigger(Ser,NOGO_CORRECT_TRIGGER)
        print(f"Correct NoGo response, trigger {NOGO_CORRECT_TRIGGER} sent")
    else:
        send_trigger(Ser,INCORRECT_TRIGGER)
        print(f"Incorrect NoGo response, trigger {INCORRECT_TRIGGER} sent")

else:
    # In case there's some unexpected image name
    print("Warning: Unknown image type, no trigger sent.")


## Building a trigger structure

### The tonic way

### The phasic way

## Adding triggers to your PsychoPy experiment

As an example, lets implement triggers in the [default Go/No-go task](https://gitlab.pavlovia.org/demos/go_nogo) that can be found on Pavlovia

### Open the port at the beginning of your experiment

### Make sure the trigger channel is set to 0 as soon as your experiment starts

### Identify the events that you need to mark, and appropriately call the function to send triggers

### End your experiment
#### Make sure the trigger channel is set back to 0 when your experiment stops
#### Close the port properly